In [ ]:
%pip install langchain_community
%pip install langchain-openai
%pip install langchainhub
%pip install chromadb
%pip install langchain
%pip install python-dotenv
%pip install PyPDF2 -q --user
%pip install rank_bm25
%pip install langchain_core

# new installs for document loaders
%pip install bs4
%pip install python-docx
%pip install docx2txt
%pip install jq

In [ ]:
import os
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import PromptTemplate
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [ ]:
# If you cannot use .env, save the file as env and use this code to access:
_ = load_dotenv(dotenv_path='env.txt')

In [ ]:
# OpenAI Setup
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
#### DOCUMENT LOADERS ####

In [ ]:
# set up our file to be available in all formats:
from bs4 import BeautifulSoup
import docx
import json

# Document paths
pdf_path = "google-2023-environmental-report.pdf"
html_path = "google-2023-environmental-report.html"
word_path = "google-2023-environmental-report.docx"
json_path = "google-2023-environmental-report.json"

with open(pdf_path, "rb") as pdf_file:
    pdf_reader = PdfReader(pdf_file)
    pdf_text = "".join(page.extract_text() for page in pdf_reader.pages)

    # Text to HTML
    soup = BeautifulSoup("<html><body></body></html>", "html.parser")
    soup.body.append(pdf_text)
    with open(html_path, "w", encoding="utf-8") as html_file:
        html_file.write(str(soup))

    # Text to Word
    doc = docx.Document()
    doc.add_paragraph(pdf_text)
    doc.save(word_path)

    # Text to JSON
    with open(json_path, "w") as json_file:
        json.dump({"text": pdf_text}, json_file)

In [ ]:
#### INDEXING ####

In [ ]:
# HTML Loader
# Other options:
# https://python.langchain.com/v0.2/docs/how_to/document_loader_html/
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader(html_path)
docs = loader.load()

In [ ]:
# PDF Loader
# Other options:
# https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/
from PyPDF2 import PdfReader

docs = []
with open(pdf_path, "rb") as pdf_file:
    pdf_reader = PdfReader(pdf_file)
    pdf_text = "".join(page.extract_text() for page in pdf_reader.pages)
    docs = [Document(page_content=page) for page in pdf_text.split("\n\n")]

In [ ]:
# Microsoft Word Loader
# Other options:
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader(word_path)
docs = loader.load()

In [ ]:
# JSON Loader
# https://python.langchain.com/v0.2/docs/how_to/document_loader_json/
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path=json_path,
    jq_schema='.text',
)

docs = loader.load()

In [ ]:
# use the same splitter for all of them:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=200
)

splits = character_splitter.split_documents(docs)

In [ ]:
splits[0]

In [ ]:
# Chroma Vector Store
chroma_client = chromadb.Client()
embedding_function = OpenAIEmbeddings()
collection_name = "google_environmental_report"

dense_documents = [Document(page_content=doc.page_content, metadata={"id": str(i), "search_source": "dense"}) for i, doc in enumerate(splits)]
sparse_documents = [Document(page_content=doc.page_content, metadata={"id": str(i), "search_source": "sparse"}) for i, doc in enumerate(splits)]

vectorstore = Chroma.from_documents(
    documents=dense_documents,
    embedding=embedding_function,
    collection_name=collection_name,
    client=chroma_client
)

In [ ]:
# Create dense retriever
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Create sparse retriever
sparse_retriever = BM25Retriever.from_documents(sparse_documents, k=10)

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[dense_retriever, sparse_retriever], weights=[0.5, 0.5], c=0, k=10)

In [ ]:
#### RETRIEVAL and GENERATION ####

In [ ]:
# Prompt
prompt = hub.pull("jclemens24/rag-prompt")

In [ ]:
# Relevance check prompt
relevance_prompt_template = PromptTemplate.from_template(
    """
    Given the following question and retrieved context, determine if the context is relevant to the question.
    Provide a score from 1 to 5, where 1 is not at all relevant and 5 is highly relevant.
    Return ONLY the numeric score, without any additional text or explanation.

    Question: {question}
    Retrieved Context: {retrieved_context}

    Relevance Score:"""
)

In [ ]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
def extract_score(llm_output):
    try:
        score = float(llm_output.strip())
        return score
    except ValueError:
        return 0

def conditional_answer(x):
    relevance_score = extract_score(x['relevance_score'])
    if relevance_score < 4:
        return "I don't know."
    else:
        return x['answer']

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | RunnableParallel(
        {"relevance_score": (
            RunnablePassthrough()
            | (lambda x: relevance_prompt_template.format(question=x['question'], retrieved_context=x['context']))
            | llm
            | StrOutputParser()
        ), "answer": (
            RunnablePassthrough()
            | prompt
            | llm
            | StrOutputParser()
        )}
    )
    | RunnablePassthrough().assign(final_answer=conditional_answer)
)

In [ ]:
rag_chain_with_source = RunnableParallel(
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
# User Query
user_query = "What are Google's environmental initiatives?"
result = rag_chain_with_source.invoke(user_query)
retrieved_docs = result['context']

print(f"Original Question: {user_query}\n")
print(f"Relevance Score: {result['answer']['relevance_score']}\n")
print(f"Final Answer:\n{result['answer']['final_answer']}\n\n")
print("Retrieved Documents:")
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"Document {i}: Document ID: {doc.metadata['id']} source: {doc.metadata['search_source']}")
    print(f"Content:\n{doc.page_content}\n")